In [1]:
from collections import namedtuple, deque
from math import ceil
import time

import numpy as np

In [2]:
Item = namedtuple("Item", ['index', 'value', 'weight'])

relax_Item = namedtuple("Item", ['index', 'value', 'weight', 'v_w_ratio', 'relaxation'])

In [3]:
with open('data/ks_4_0', 'r') as input_data_file:
    input_data = input_data_file.read()

In [4]:
lines = input_data.split('\n')

firstLine = lines[0].split()
item_count = int(firstLine[0])
capacity = int(firstLine[1])

items = []
relax_items = []

for i in range(1, item_count+1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i-1, int(parts[0]), int(parts[1])))
    relax_items.append(relax_Item(i-1, int(parts[0]), int(parts[1]), round(int(parts[0])/int(parts[1]), 2), 0))

In [5]:
capacity

11

In [6]:
item_count

4

In [7]:
items

[Item(index=0, value=8, weight=4),
 Item(index=1, value=10, weight=5),
 Item(index=2, value=15, weight=8),
 Item(index=3, value=4, weight=3)]

In [8]:
def knapsack_dp(items, capacity):
    matrix = np.zeros([len(items)+1,capacity+1])
    
    for i in range(1, len(items)+1):
        for c in range(capacity+1):
            if items[i-1].weight > c:
                matrix[i][c] = matrix[i-1][c]
            else:
                matrix[i][c] = max(matrix[i-1][c], matrix[i-1][c-items[i-1].weight]+items[i-1].value)

    return matrix

In [9]:
def reconstruct_knapsack(matrix, items):
    remain_capacity = capacity
    index_list = [0] * len(items)
    for i in range(0, len(items))[::-1]:
        if (items[i].weight <=  remain_capacity) and \
        (matrix[i][int(remain_capacity-items[i].weight)]+items[i].value >= matrix[i][int(remain_capacity)]):
            index_list[items[i].index] = 1
            remain_capacity -= items[i].weight
        else:
            pass

    return index_list

In [10]:
start_time = time.time()

matrix = knapsack_dp(items, capacity)
index_list = reconstruct_knapsack(matrix, items)

value = matrix[-1][-1]

end_time = time.time()

print("This took %.2f seconds" % (end_time - start_time))

This took 0.00 seconds


In [11]:
output_data = str(value) + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, index_list))

In [12]:
output_data

'19.0 0\n0 0 1 1'

branch and bound

In [13]:
relax_items = sorted(relax_items, key=lambda x: x.v_w_ratio)[::-1]

In [14]:
State = namedtuple("State", ['position', 'value', 'free_capacity', 'estimate', 'items'])

In [15]:
def linear_relaxation(items, cap, update=False, max_value=0):
    solved = False
    n_items = len(items)
        
    for n, i in enumerate(items):
        if (cap - i.weight) >= 0:

            cap -= i.weight
            max_value += i.value

            if update:
                items[n] = items[n]._replace(relaxation = 1)
                
            if n == n_items-1:
                solved = True

            if cap == 0:
                solved = True
                break

        else:
            ratio = round((cap / i.weight),2)
            max_value += ceil(i.value*ratio)
            
            if update:
                items[n] = items[n]._replace(relaxation = ratio)

            break
    
    return solved, items, max_value

In [16]:
def dfs_branch_bound(items, capacity):
    
    solved, items, init_estimate = linear_relaxation(items, capacity, True)
    
    init_state = State(0, 0, capacity, init_estimate, ())
    solution = State(0, 0, capacity, init_estimate, ())

    stack = deque([init_state])
    n_items = len(items)

    while stack:

        current_state = stack.pop()
        pos = current_state.position
        val = current_state.value
        cap = current_state.free_capacity
        estimate = current_state.estimate
        item_tuple = current_state.items

        #if next item is not choosen:
        if pos >= n_items:
            continue

        _, _, estimate = linear_relaxation(items[pos:], cap, False, val)

        if solution.value > estimate:
            continue

        new_state = State(pos+1, val, cap, estimate, item_tuple)
        stack.append(new_state)

        cap -= items[pos].weight

        if cap >= 0:

            val += items[pos].value
            item_tuple += (items[pos].index,)
            new_state = State(pos+1, val, cap, estimate, item_tuple)
            stack.append(new_state)
            
            if val > solution.value:
                solution = new_state
                
    return solution

In [17]:
best = dfs_branch_bound(relax_items, capacity)
best

State(position=4, value=19, free_capacity=0, estimate=19, items=(2, 3))

In [25]:
index_list = [0]*len(relax_items)
value = best.value

In [26]:
for i in best.items:
    index_list[relax_items[i].index] = 1

In [27]:
output_data = str(value) + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, index_list))